In [8]:
import sys
sys.path.append('../')
from  mongo.lib.mongo_db  import db_conn
import pandas as pd
import numpy as np
from pyecharts import Bar,configure,Pie
import time
from es_select import ES
import itertools


In [9]:
def date_to_time(date):
    timeArray = time.strptime(date, "%Y%m%d")
    time_semp = time.mktime(timeArray)
    return time_semp   # 2013
def date_to_date(date):
    timeArray = time.strptime(date, "%a %b %d %H:%M:%S %Y")
    time_semp = time.strftime("%Y%m%d%H%M%S", timeArray);
    return time_semp   # 2013

def time_to_today(timeStamp):
    timeArray = time.localtime(int(timeStamp))
    timeArray =time.strftime("%Y-%m-%d %H:%M:%S",timeArray)
    return timeArray   # 2013

## 1、查询20181106日联通出现 5017 （尊敬的客户您好，系统检测您的访问过于频繁，为保障您的账户安全请明天再试。）

In [10]:
data=db_conn.sid_info.find({"end_time":{'$gte':date_to_time('20181106'),'$lte':date_to_time('20181107')},'status':5017})
data1=list(data)

In [11]:
data1_rpt_pd=pd.DataFrame(x for x in data1)

## 2、将代理信息和目标URL取出，统计请求次数、使用代理数量、代理去重列表

In [12]:
def get_proxy(data1_rpt_pd):
    all_sid_http_proxy=[]
    for sid in data1_rpt_pd.sid.values.tolist():
        es=ES('20181107')
        one_sid_http_proxy={}
        #print (sid)
        ret_list=es.get_all_ret(sid)
        #print(len(ret_list))
        one_sid_http_proxy['sid']=sid
        one_sid_http_proxy['tel']=data1_rpt_pd.loc[data1_rpt_pd.sid.values==sid,'tel'].values.tolist()[0]
        one_sid_http_proxy['status']=data1_rpt_pd.loc[data1_rpt_pd.sid.values==sid,'status'].values.tolist()[0]
        one_sid_http_proxy['http_proxy']=[]
        one_sid_http_proxy['url_base']=[]
        one_sid_http_proxy['dates']=[]
        for i,one_http in enumerate(ret_list):
            one_sid_http_proxy['http_proxy'].append(one_http['_source']['MSG']['proxies']['http'].replace('http://','').split('.')[0])
            one_sid_http_proxy['url_base'].append(one_http['_source']['MSG']['url'])
            one_sid_http_proxy['dates'].append(date_to_date(one_http['_source']['DATETIME']))
        l = [(k, len(list(g))) for k, g in itertools.groupby(one_sid_http_proxy['http_proxy'])]
        one_sid_http_proxy['http_req_count']=l
        one_sid_http_proxy['req_count']=len(ret_list)
        one_sid_http_proxy['http_unique']=list(set(one_sid_http_proxy['http_proxy']))
        one_sid_http_proxy['http_unique_count']=len(one_sid_http_proxy['http_unique'])
        all_sid_http_proxy.append(one_sid_http_proxy)
    return all_sid_http_proxy
        

In [13]:
all_sid_http_proxy=get_proxy(data1_rpt_pd)

In [14]:
all_sid_http_proxy_pd=pd.DataFrame(x for x in all_sid_http_proxy) 

In [15]:
all_sid_http_proxy_pd=all_sid_http_proxy_pd.fillna(0)

## 3、将数据按照手机号码组织，并获取单个客户的请求此次，使用的代理数量

In [16]:
def chain_l(l):
    ret_dict={}
    for one in l:
        if one[0] in ret_dict:
            ret_dict[one[0]]+=one[1]
        else:
            ret_dict[one[0]]=one[1]
    return ret_dict

In [17]:
import itertools
def get_new_tel_list(all_sid_http_proxy_pd):
    new_tel_list=[]
    for tel in all_sid_http_proxy_pd.tel.unique().tolist():
        new_tel_dict={}
        tmp=all_sid_http_proxy_pd.loc[all_sid_http_proxy_pd.tel.values==tel,['tel','req_count','http_req_count','http_unique_count','http_unique']]
        tmp=tmp.fillna(0)
        #print(tmp.shape)
        new_tel_dict['tel']=tel
        new_tel_dict['req_count']=tmp.req_count.values.sum()
        #print(tmp.http_req_count.values.tolist())
        l=list(itertools.chain(*tmp.http_req_count.values.tolist()))
        l_dict=chain_l(l)
        new_tel_dict['http_req_count']=tuple(l_dict.values())
        new_tel_dict['http_unique_count']=tmp.http_unique_count.values.sum()
        new_tel_dict['http_unique']= tuple(l_dict)
        new_tel_list.append(new_tel_dict)
    new_tel_pd=pd.DataFrame(x for x in new_tel_list)
    return new_tel_pd

In [18]:
new_tel_pd=get_new_tel_list(all_sid_http_proxy_pd)

## 4、当天只使用同一个代理但是被封的客户

In [19]:
filte=np.logical_and(new_tel_pd.http_unique_count.values==1,True)
tmp_1=new_tel_pd.loc[filte,['tel','req_count','http_unique_count','http_unique']]

In [20]:
tmp_1

,tel,req_count,http_unique_count,http_unique
2,18665287903,329,1,"(squidsz108,)"
5,18685282281,134,1,"(squidsz75,)"
6,15505996669,173,1,"(squidsz35,)"
19,15172525372,13,1,"(squidsz34,)"
21,18685432568,128,1,"(squidsz97,)"
24,18272115317,13,1,"(squidsz94,)"
25,15854686844,13,1,"(squidsz18,)"
26,18685378880,125,1,"(squidsz13,)"
29,15519044333,128,1,"(squidsz19,)"
30,18671666468,124,1,"(squidsz84,)"


### 发现  15172525372 ,15854686844 和 18272115317 存在异常值

In [21]:
data=db_conn.sid_info.find({"end_time":{'$gte':date_to_time('20181106'),'$lte':date_to_time('20181107')},'tel':{'$in':['15172525372','18272115317','15854686844']}})
data2=list(data)
data2=pd.DataFrame(x for x in data2)
one_sid_http_proxy=get_proxy(data2)
one_tel_http_proxy_pd=pd.DataFrame(x for x in one_sid_http_proxy) 
one_tel_http_proxy_pd=get_new_tel_list(one_tel_http_proxy_pd)

In [22]:
one_tel_http_proxy_pd.loc[:,['tel','req_count','http_unique_count','http_unique']]

,tel,req_count,http_unique_count,http_unique
0,15172525372,13,1,"(squidsz34,)"
1,15854686844,13,1,"(squidsz18,)"
2,18272115317,13,1,"(squidsz94,)"


### 以上三个请求均为移动

### 接下来分析 代理squidsz34，在这次请求前后是否正常

In [24]:
es=ES('20181107')
proxy_ret=es.get_proxy_ret('http://squidsz34.dianhua.cn:8080')
del es

In [25]:
one_sid_http_proxy_list=[]
for one in proxy_ret:
    one_sid_http_proxy={}
    one_sid_http_proxy['http_proxy']='http://squidsz34.dianhua.cn:8080'
    one_sid_http_proxy['url_base']=(one['_source']['MSG']['url'])
    one_sid_http_proxy['dates']=(date_to_date(one['_source']['DATETIME']))
    one_sid_http_proxy['sid']=((one['_source']['MSG']['sid']))
    one_sid_http_proxy_list.append(one_sid_http_proxy)


In [26]:
def sid_get_err_code(sid):
    data=db_conn.sid_info.find_one({"sid":sid,},{'_id':0,'status':1})
    return data.get('status',1)

In [27]:
one_sid_http_proxy_pd=pd.DataFrame(x for x in one_sid_http_proxy_list)
#one_sid_http_proxy_pd.groupby('sid')['http_proxy'].size().reset_index()
ip_proxy_sort_pd=one_sid_http_proxy_pd.groupby('sid')['dates'].max().reset_index().sort_values(by="dates")
ip_proxy_sort_pd['status']=ip_proxy_sort_pd.sid.map(sid_get_err_code)

In [28]:
ip_proxy_sort_pd

,sid,dates,status
46,SIDecafae786bda4c7b899abfe39aeb796f,20181106001905,0
42,SIDd09fb89a221b41df92371b6a6edb7667,20181106061904,0
7,SID16e6e652939249bcba76d4b13ba1b735,20181106074014,0
17,SID44d43d958fd84187904dd54eeb0ed868,20181106090805,0
26,SID72a869b8b7b74f2bb24b8346dd78a5ff,20181106091217,0
32,SIDa8e84ad3be1744199dbb1b8cb4035df5,20181106095507,0
34,SIDb43a869505dc47f68b171d21f47fa1b4,20181106100309,0
21,SID57d164fe490045629344ac45a54c5f3b,20181106101921,0
2,SID09de2d93f8fe4d348a712ff7b76eb583,20181106104220,0
25,SID714c660bcd4045e99efaf32d1a9d92a5,20181106105327,5017


#### 出现了请求频繁，该代理之后的其他会话中还是可以正常使用，并爬取成功，怀疑可能该客户当天通过其他渠道操作过账户。

## 5、使用多个代理被封情况分析
req_count 为请求官网次数

http_req_count 为http_unique中代理对应请求次数

http_unique_count 为使用代理个数

http_unique 为被使用过的代理

### A、使用代理数量超过1个的客户

In [29]:
filte=np.logical_and(new_tel_pd.http_unique_count.values>1,True)
tmp_2=new_tel_pd.loc[filte,['tel','req_count','http_req_count','http_unique_count','http_unique']]
tmp_2

,tel,req_count,http_req_count,http_unique_count,http_unique
0,18649764213,15,"(3, 2, 2, 3, 4, 1)",6,"(squidsz47, squidsz44, squidsz62, squidsz43, s..."
1,18580698151,190,"(128, 62)",2,"(squidsz12, squidsz15)"
3,18685794878,151,"(15, 136)",2,"(squidsz124, squidsz122)"
4,18507450873,17,"(3, 3, 2, 3, 1, 1, 1, 1, 1, 1)",10,"(squidsz25, squidsz30, squidsz28, squidsz49, s..."
7,15576882298,14,"(8, 1, 1, 1, 1, 1, 1)",7,"(squidsz04, squidsz30, squidsz13, squidsz14, s..."
8,18138764589,20,"(5, 5, 5, 5)",4,"(squidsz68, squidsz65, squidsz74, squidsz11)"
9,15963750395,26,"(13, 13)",2,"(squidsz105, squidsz111)"
10,13277091651,6,"(1, 1, 1, 1, 1, 1)",6,"(squidsz38, squidsz33, squidsz88, squidsz86, s..."
11,13042237688,161,"(7, 2, 19, 132, 1)",5,"(squidsz67, squidsz71, squidsz73, squidsz35, s..."
12,13192028199,32,"(2, 7, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2)",13,"(squidsz62, squidsz75, squidsz73, squidsz49, s..."


### B、分析代理总调用次数小于100次的客户

In [34]:
filte=np.logical_and(new_tel_pd.http_unique_count.values>1,new_tel_pd.req_count.values<100)
tmp_3=new_tel_pd.loc[filte,['tel','req_count','http_req_count','http_unique_count','http_unique']]
tmp_3

,tel,req_count,http_req_count,http_unique_count,http_unique
0,18649764213,15,"(3, 2, 2, 3, 4, 1)",6,"(squidsz47, squidsz44, squidsz62, squidsz43, s..."
4,18507450873,17,"(3, 3, 2, 3, 1, 1, 1, 1, 1, 1)",10,"(squidsz25, squidsz30, squidsz28, squidsz49, s..."
7,15576882298,14,"(8, 1, 1, 1, 1, 1, 1)",7,"(squidsz04, squidsz30, squidsz13, squidsz14, s..."
8,18138764589,20,"(5, 5, 5, 5)",4,"(squidsz68, squidsz65, squidsz74, squidsz11)"
9,15963750395,26,"(13, 13)",2,"(squidsz105, squidsz111)"
10,13277091651,6,"(1, 1, 1, 1, 1, 1)",6,"(squidsz38, squidsz33, squidsz88, squidsz86, s..."
12,13192028199,32,"(2, 7, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2)",13,"(squidsz62, squidsz75, squidsz73, squidsz49, s..."
13,13096871866,10,"(3, 3, 3, 1)",4,"(squidsz34, squidsz63, squidsz43, squidsz70)"
17,18503653888,15,"(3, 4, 1, 1, 1, 1, 1, 1, 1, 1)",10,"(squidsz70, squidsz68, squidsz19, squidsz20, s..."
18,18609558955,14,"(3, 2, 3, 3, 1, 1, 1)",7,"(squidsz09, squidsz12, squidsz15, squidsz18, s..."


#### 查看这些客户当日是否有其他返回码的请求

In [31]:
data=db_conn.sid_info.find({"end_time":{'$gte':date_to_time('20181106'),'$lte':date_to_time('20181107')},'tel':{'$in':tmp_3.tel.values.tolist()}})
data2=list(data)
data2=pd.DataFrame(x for x in data2)
some_sid_http_proxy=get_proxy(data2)
some_sid_http_proxy_pd=pd.DataFrame(x for x in some_sid_http_proxy) 
some_sid_http_proxy_pd=get_new_tel_list(some_sid_http_proxy_pd)

In [32]:
some_sid_http_proxy_pd

,http_req_count,http_unique,http_unique_count,req_count,tel
0,"(3, 3, 3, 1)","(squidsz34, squidsz63, squidsz43, squidsz70)",4,10,13096871866
1,"(3, 3, 2, 3, 3, 1, 1, 1)","(squidsz95, squidsz97, squidsz111, squidsz114,...",8,17,13103890371
2,"(3, 1, 1)","(squidsz43, squidsz50, squidsz46)",3,5,13189238185
3,"(2, 7, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2)","(squidsz62, squidsz75, squidsz73, squidsz49, s...",13,32,13192028199
4,"(2, 2, 5, 1, 1)","(squidsz86, squidsz88, squidsz91, squidsz93, s...",5,11,13195877164
5,"(1, 1, 1, 1, 1, 1)","(squidsz38, squidsz33, squidsz88, squidsz86, s...",6,6,13277091651
6,"(5, 2, 2, 1, 1)","(squidsz19, squidsz18, squidsz17, squidsz22, s...",5,11,13294780204
7,"(4, 4)","(squidsz06, squidsz07)",2,8,13722666541
8,"(8, 1, 1, 1, 1, 1, 1)","(squidsz04, squidsz30, squidsz13, squidsz14, s...",7,14,15576882298
9,"(13, 13)","(squidsz105, squidsz111)",2,26,15963750395


#### 这些用户当日不存在其他请求，由此推测可能是频繁变更IP地址引起封禁

### C、分析请求数量超过100的代理

In [35]:
filte=np.logical_and(new_tel_pd.http_unique_count.values>1,new_tel_pd.req_count.values>100)
tmp_4=new_tel_pd.loc[filte,['tel','req_count','http_req_count','http_unique_count','http_unique']]
tmp_4

,tel,req_count,http_req_count,http_unique_count,http_unique
1,18580698151,190,"(128, 62)",2,"(squidsz12, squidsz15)"
3,18685794878,151,"(15, 136)",2,"(squidsz124, squidsz122)"
11,13042237688,161,"(7, 2, 19, 132, 1)",5,"(squidsz67, squidsz71, squidsz73, squidsz35, s..."
14,13118507914,140,"(2, 14, 2, 122)",4,"(squidsz45, squidsz65, squidsz41, squidsz39)"
15,15585883123,141,"(3, 138)",2,"(squidsz130, squidsz16)"
16,18585210620,183,"(15, 15, 15, 138)",4,"(squidsz97, squidsz71, squidsz06, squidsz128)"


#### 使用代理数量相对B情况要少，但是均存在某一个代理使用超过100的情况，可能是因为当日请求量超过阈值或者Ip地址发送变化引起

### D、分析C中代理使用量超过100的情况发生的时机

In [36]:
def select_tel(tel):
    data=db_conn.sid_info.find({"end_time":{'$gte':date_to_time('20181106'),'$lte':date_to_time('20181107')},'tel':tel})
    data2=list(data)
    data2=pd.DataFrame(x for x in data2)
    some_sid_http_proxy=get_proxy(data2)
    some_sid_http_proxy_pd=pd.DataFrame(x for x in some_sid_http_proxy) 
    some_sid_http_proxy_pd=some_sid_http_proxy_pd.loc[some_sid_http_proxy_pd.req_count.values>0,['sid','req_count','tel','status','dates']]
    some_sid_http_proxy_pd['dates']=some_sid_http_proxy_pd.dates.map(max)
    return some_sid_http_proxy_pd

In [37]:
select_tel('18580698151')

,sid,req_count,tel,status,dates
0,SIDf7f785e67639491cb1bd8319cf33060a,128,18580698151,5017,20181106224212
1,SID91066c1673d74ce6a2135da4fea438f7,62,18580698151,5017,20181106224207


In [38]:
select_tel('13042237688')

,sid,req_count,tel,status,dates
0,SID22914fa2f81f4839ab44fc580044bc97,133,13042237688,5017,20181106165850
1,SID4dfe4f17b47f497f98020164b919757c,28,13042237688,5017,20181106170005


#### 由此可以看出，13042237688 先发生了133次官网请求，然后重试有发生了一个28次的官网请求，在首次请求时已经触发封禁

## 5、结论
### 1）、单次爬虫事务中，如果短时间内请求官网超过125次，可能被封禁
### 2）、多次爬虫事务中，如果使用多个ip代理请求官网，超过2个ip将可能被封禁
### 3）、联通反爬策略为，tel+ip进行识别，相同ip不同客户、相互不干扰
### 4）、针对分析结果，建立爬虫代理池对该以上反爬策略无效